# Project: NIP
# Subroject: Sylhet Floods, June 2022
## Script: fracArea_timeSeries_modifiedDevries.ipynb
This script generates a time series of the fractional inundated area, from the outputs of the modified DeVries Sentinel-1 surface water mapping algorithm (Thomas et al., 2021).

In [1]:
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.mask import mask
from itertools import chain
from pyproj import Transformer
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
from datetime import datetime
import geopandas as gpd
import copy
import collections
from shapely.geometry import mapping
import pycrs
from shapely.geometry import Polygon
from functools import reduce

In [2]:
# Set the root path
rootPath = Path('Z:/media/mule/Projects/NASA/NIP/Data')

In [3]:
# Add path for the Helpers modules
module_path = os.path.abspath(os.path.join('C:/Users/alexsaunders/Documents/01_uoa/04_git/NIP/Sylhet/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
# Import module from analyse_modifiedDevries containing helpful functions to use
import importlib
import Helpers.analyse_modifiedDevries as analyse_DV
importlib.reload(analyse_DV)

<module 'Helpers.analyse_modifiedDevries' from 'C:\\Users\\alexsaunders\\Documents\\01_uoa\\04_git\\NIP\\Sylhet\\Helpers\\analyse_modifiedDevries.py'>

## PART 1: Define the ROI(s) for computing fractional flooded area
Load in the admin areas and gauge locations, create some buffer sizes around the gauges to use as ROI for computing FFA

In [6]:
inputPath = 'Raster/Sylhet/Sen1MitchellSingleOrbit/dryBaseV2/Mosaic'
mosaics = [file for file in (rootPath/inputPath).iterdir() if file.is_file() and '2022' in str(file)]
mosaics

[WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/Sylhet/Sen1MitchellSingleOrbit/dryBaseV2/Mosaic/sylhet_dist_20220501_43024.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/Sylhet/Sen1MitchellSingleOrbit/dryBaseV2/Mosaic/sylhet_dist_20220511_43163.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/Sylhet/Sen1MitchellSingleOrbit/dryBaseV2/Mosaic/sylhet_dist_20220513_43199.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/Sylhet/Sen1MitchellSingleOrbit/dryBaseV2/Mosaic/sylhet_dist_20220523_43338.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/Sylhet/Sen1MitchellSingleOrbit/dryBaseV2/Mosaic/sylhet_dist_20220525_43374.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/Sylhet/Sen1MitchellSingleOrbit/dryBaseV2/Mosaic/sylhet_dist_20220604_43513.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/Sylhet/Sen1MitchellSingleOrbit/dryBaseV2/Mosaic/sylhet_dist_20220606_43549.tif'),
 WindowsPath('Z:/med

In [7]:
# Import the ROI geometry from shapefile
adminShapePath = rootPath/'Shapefiles/AdminHDX'
districts = gpd.read_file(adminShapePath/'bgd_admbnda_adm2_bbs_20201113.shp')
upazillas = gpd.read_file(adminShapePath/'bgd_admbnda_adm3_bbs_20201113.shp')
sylhet_dist = districts[districts['ADM2_EN']=='Sylhet']
sylhet_upa = upazillas[upazillas['ADM3_EN']=='Sylhet Sadar']

In [8]:
# Or instead create the ROI as a buffer around the gauge locations

# Import the gauge station locations
gaugePath = rootPath/'Stream_Gauge/StreamGaugeShapefiles'
gaugeLocs = gpd.read_file(gaugePath/'streamgaugew_division.shp') # read the file as a geopandas

# Create buffer around all gauges - use several different buffer sizes starting from 100 m, to 1 km
gaugeLocsGeoms = gaugeLocs.geometry
buffer_sizes = [5, 10, 15]#[0.1, 0.2, 0.5, 1]
pd.options.mode.chained_assignment = None  # default='warn'

for buffer_size in buffer_sizes:
    
    # Add new field to locs geodataframe to store the buffer geometries
    gaugeLocs['buffer_'+str(buffer_size)] = 0

    # For each gaugeLoc, get the buffer and add back to the geodataframe
    for i in range(0, len(gaugeLocs)):
        gaugeLocGeom = gaugeLocsGeoms[i]
        buffer = analyse_DV.geodesic_point_buffer(gaugeLocGeom.y, gaugeLocGeom.x, buffer_size) # lat, lon, buffer (in km)
        gaugeLocs['buffer_'+str(buffer_size)][i] = Polygon(buffer)

In [9]:
# Define the gauges in Sylhet District
streamGaugeDataPath = rootPath/'Stream_Gauge/'
gauges_in_sylhet = pd.read_csv(streamGaugeDataPath/'streamgauges_SylhetDist.csv', header=0)
gauges_in_sylhet = list(gauges_in_sylhet['streamvect'])
gaugeDataSummaryMaster = pd.read_csv(streamGaugeDataPath/'CombinedAll2004-2022/Combined/summary.csv', header=0)
gauges = [match for match in gauges_in_sylhet if match in list(gaugeDataSummaryMaster['stationID'][np.logical_and(~np.isnan(gaugeDataSummaryMaster['danger_level']),gaugeDataSummaryMaster['danger_level']<1000)])]
print(gauges)
gauges_in_sylhet_geom = gaugeLocs[gaugeLocs['streamvect'].isin(gauges_in_sylhet)]
gauges_geom = gaugeLocs[gaugeLocs['streamvect'].isin(gauges)]

['SW172.5', 'SW173', 'SW175.5', 'SW251', 'SW266', 'SW267']


In [10]:
gauges_list = list(chain.from_iterable([list(gauges_geom['streamvect']+'_'+str(buffer_size)+'km') for buffer_size in buffer_sizes]))
gauges_geom_list = list(chain.from_iterable([list(gauges_geom['buffer_'+str(buffer_size)]) for buffer_size in buffer_sizes]))

In [11]:
# Create a single geodataframe of ROIs over which to calculate FFA
rois = {'id': ['Sylhet_District', 'Sylhet_Upazilla']+gauges_list, 
        'geometry': list(sylhet_dist.geometry)+list(sylhet_upa.geometry)+gauges_geom_list}
rois_gdf = gpd.GeoDataFrame(rois, geometry='geometry', crs='epsg:4326')

## PART 2: Compute the fractional inundated area at each timestep
For each date, extract the fractional inundated area for the specified ROI.

In [13]:
# Get the dates of images
dates = [str(datetime.strptime(mosaic.stem[-14:-6], '%Y%m%d').date()) for mosaic in mosaics]
print(dates)

['2022-05-01', '2022-05-11', '2022-05-13', '2022-05-23', '2022-05-25', '2022-06-04', '2022-06-06', '2022-06-16', '2022-06-18', '2022-06-28', '2022-06-30', '2022-07-12', '2022-07-22', '2022-07-24', '2022-08-03', '2022-08-05', '2022-08-15', '2022-08-27', '2022-08-29']


In [216]:
# Run the function to create temp raster file for using in masking - ONLY NEEDS TO BE RUN ONCE

# Run the function which creates a raster of values 10, the same dimensions as the flood raster, to be used for clipping to the ROI later
# date1 = '20220604'
# mosaic = [mosaic for mosaic in mosaics if date1 in str(mosaic)][0]
# analyse_DV.create_temp_raster_for_mask(mosaic, rootPath, inputPath) # creates raster_tmp

In [15]:
# Load the temp raster in and intersect it with the ROI - this will be used for clipping to the ROI since it contains values of 10
raster_tmp = rasterio.open(rootPath/'Raster/Sylhet/Sen1MitchellSingleOrbit/Mosaic'/'temp.tif') # can recreate this file if needs for a different ROI area by rerunning the above function

In [16]:
# Run the function to get the fractional flooded area for all image dates, for all the provided ROIs
mosaic_dates = []
pctWaters = []
rois_list = []

# Loop through image dates
for n, mosaic in enumerate(mosaics):
    
    # Get the date
    mosaic_date = str(datetime.strptime(mosaic.stem[-14:-6], '%Y%m%d').date())
    print('Mosaic:', mosaic_date, mosaic)

#     # Load the raster using rioxarray
#     raster = prep_raster.load_raster_for_comparison(mosaic, -10)

#     # Reproject and match the GFM raster to the modified Devries raster to make the processing the same
#     raster_reproj = prep_raster.reproj_match_raster(raster, raster_Devries)
    
    # Load the raster
    with rasterio.open(mosaic) as file:
        raster = file.read(1)

    # Set the water values to 1
    water = copy.copy(raster)
    water[water==1]=1
    
    # Loop through all ROI geometries
    for j in range(len(rois_gdf)):

        # Run the function to create the raster mask for the ROI, using the temp raster containing all values 10
        roiID = rois_gdf.id[j]
        roi = rois_gdf.loc[[j]]
        roi_mask = analyse_DV.create_roi_mask(raster_tmp, roi)

        # Apply the mask to the water_sum raster by performing an array sum operation
        water_roi = np.add(water, roi_mask)

        # Get the number of pixels of water for each date for the ROI only (pixel values in ROI have been shifted by +10 from original values)
        water_roi_vals = list(chain.from_iterable(water_roi.tolist()))
        counter_roi = collections.Counter(water_roi_vals) # 10 = none, 11 = water, other = invalid or outside ROI
        pctWater_roi = counter_roi[11] / (counter_roi[10] + counter_roi[11])
        #print(counter_roi)
        print('ROI % water at {0} for {1}: {2:2.1f}%'.format(mosaic_date, roiID, pctWater_roi*100))
        
        # Record the results for the date and ROI
        mosaic_dates.append(mosaic_date)
        rois_list.append(roiID)
        pctWaters.append(pctWater_roi)

Mosaic: 2022-05-01 Z:\media\mule\Projects\NASA\NIP\Data\Raster\Sylhet\Sen1MitchellSingleOrbit\dryBaseV2\Mosaic\sylhet_dist_20220501_43024.tif
ROI % water at 2022-05-01 for Sylhet_District: 13.2%
ROI % water at 2022-05-01 for Sylhet_Upazilla: 11.2%
ROI % water at 2022-05-01 for SW172.5_5km: 17.0%
ROI % water at 2022-05-01 for SW173_5km: 6.4%
ROI % water at 2022-05-01 for SW175.5_5km: 9.3%
ROI % water at 2022-05-01 for SW251_5km: 37.6%
ROI % water at 2022-05-01 for SW266_5km: 15.0%
ROI % water at 2022-05-01 for SW267_5km: 8.9%
ROI % water at 2022-05-01 for SW172.5_10km: 18.1%
ROI % water at 2022-05-01 for SW173_10km: 7.9%
ROI % water at 2022-05-01 for SW175.5_10km: 8.6%
ROI % water at 2022-05-01 for SW251_10km: 22.7%
ROI % water at 2022-05-01 for SW266_10km: 17.4%
ROI % water at 2022-05-01 for SW267_10km: 8.5%
ROI % water at 2022-05-01 for SW172.5_15km: 16.4%
ROI % water at 2022-05-01 for SW173_15km: 10.0%
ROI % water at 2022-05-01 for SW175.5_15km: 7.1%
ROI % water at 2022-05-01 for SW2

In [17]:
# Create dataframe of the results - fractional flooded area
fracFloodArea = pd.DataFrame(data = [mosaic_dates, rois_list, pctWaters], index = ['Date','ROI','FFA']).T
fracFloodArea = fracFloodArea.pivot(index='Date', columns='ROI', values='FFA')
fracFloodArea.index = pd.to_datetime(fracFloodArea.index)

In [19]:
# Export the results as csv
outputPath = rootPath/'Table/SylhetFracFloodedArea'
outputPath.mkdir(exist_ok=True)
pd.DataFrame.to_csv(fracFloodArea, outputPath/'FFA_sylhet_allROI_allGauges_modDevries_dryBaseV2.csv', index=True)

In [132]:
# If required, read back in from csv table
# fracFloodArea = pd.read_csv(rootPath/'Table/SylhetFracFloodedArea/FFA_sylhet_allROI_allGauges_modDevries.csv', index_col='Date')
# fracFloodArea.index = pd.to_datetime(fracFloodAreaBuffers.index)

In [19]:
# OLD VERSION
# # Run the function to get the fractional flooded area for all image dates, for the specific ROI
# # note that raster_tmp will need to be recreated if using images with a different extent (but otherwise does not need to be recreated)
# fracFloodArea_sylhetDist = analyse_DV.get_fracArea_timeSeries(mosaics, raster_tmp, sylhet_dist)
# fracFloodArea_sylhetUpa = analyse_DV.get_fracArea_timeSeries(mosaics, raster_tmp, sylhet_upa)

In [24]:
fracFloodArea

ROI,SW172.5_10km,SW172.5_15km,SW172.5_5km,SW173_10km,SW173_15km,SW173_5km,SW175.5_10km,SW175.5_15km,SW175.5_5km,SW251_10km,SW251_15km,SW251_5km,SW266_10km,SW266_15km,SW266_5km,SW267_10km,SW267_15km,SW267_5km,Sylhet_District,Sylhet_Upazilla
Date,,,,,,,,,,,,,,,,,,,,
2022-05-01,0.188821,0.168748,0.18147,0.080356,0.099973,0.062999,0.12927,0.096697,0.122095,0.284298,0.223065,0.478371,0.21624,0.187081,0.20922,0.103938,0.10634,0.089577,0.14711,0.138554
2022-05-11,0.331973,0.310239,0.328489,0.224426,0.254058,0.139409,0.167353,0.14745,0.172652,0.302748,0.287258,0.30663,0.342769,0.296451,0.380062,0.246909,0.282718,0.162206,0.265766,0.19299
2022-05-13,0.530672,0.480841,0.481612,0.44649,0.477495,0.417769,0.362789,0.3578,0.354101,0.614439,0.551612,0.6085,0.604393,0.495921,0.61166,0.360022,0.446461,0.204524,0.535057,0.399904
2022-05-23,0.516368,0.495572,0.441686,0.580296,0.575373,0.625107,0.558148,0.56431,0.596191,0.538723,0.518907,0.429315,0.649214,0.527393,0.665491,0.399863,0.497226,0.23229,0.612228,0.469083
2022-05-25,0.476244,0.464078,0.389146,0.5352,0.541294,0.574364,0.526874,0.542996,0.570982,0.49677,0.481602,0.358998,0.586924,0.485634,0.576946,0.365322,0.459594,0.206493,0.578989,0.43354
2022-06-04,0.494719,0.468541,0.436417,0.484182,0.502245,0.49412,0.548103,0.543985,0.580574,0.568443,0.522573,0.49114,0.595762,0.485998,0.593806,0.326798,0.437854,0.174323,0.561319,0.396336
2022-06-06,0.527251,0.509869,0.440406,0.551667,0.557232,0.562047,0.558456,0.569515,0.602942,0.625269,0.567457,0.59996,0.672109,0.541962,0.681323,0.387384,0.495153,0.213548,0.61308,0.463395
2022-06-16,0.488461,0.471008,0.41382,0.514787,0.518315,0.514023,0.557149,0.56481,0.569033,0.652664,0.586016,0.64947,0.665203,0.530578,0.672554,0.374408,0.477063,0.177602,0.600541,0.483848
2022-06-18,0.595189,0.561515,0.556837,0.605004,0.608784,0.623588,0.656905,0.656345,0.660165,0.662773,0.595337,0.654544,0.690529,0.563425,0.69764,0.438416,0.539048,0.249083,0.657181,0.493217
